In [1]:
import pickle
import gzip
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
#获取fashion数据集并对数据集进行reshape操作
def load_data():
    dataset_path=Path('G:/jupyter/MNIST')
    fashion_path=dataset_path/'fashion.pkl.gz'
    f = gzip.open(fashion_path, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    #解压后的文件夹有训练集，验证集，测试集三部分
    f.close()
    return (training_data, validation_data, test_data)
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    #获取三部分
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    #对于input输入（也就是image），改为784*1的大小
    training_results = [vectorized_result(y) for y in tr_d[1]]
    #对于标签output，进行调整
    training_data = zip(training_inputs, training_results)
    #新的训练集数据
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    #对验证集和测试集进行类似操作
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    #返回新的三部分
    return (training_data, validation_data, test_data)
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [2]:
training_data, validation_data, test_data=load_data_wrapper()
training_data=list(training_data)

In [3]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        #这一部分是初始化各种值，num_layers表示有几层神经网络
        #size就是输入的
        #阈值（偏移量）是根据输入值随机设置
        #权重是根据输入层数目和输出层数目随机决定
        self.num_layers = len(sizes)
        print(self.num_layers)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        #向前传播——也就是对每一个输出的值进行激活，a表示输出值
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        #模型建立小批次处理

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        cost=[]
        #反向传播函数
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        #激活函数设置(初始值是x)
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            #矩阵相乘（z=w*x+b）
            zs.append(z)
            activation = sigmoid(z)
            #激活后的输出，也就是下一层的输入
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations - y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [4]:
net = Network([784, 50, 10])

3


In [ ]:
net = Network([784, 50, 10])
net.SGD(training_data,30, 10, 0.1, test_data=validation_data)

In [ ]:
net.SGD(training_data,40, 10, 0.1, test_data=validation_data)

In [ ]:
net.SGD(training_data,40, 10, 0.01, test_data=test_data)

In [18]:
net.SGD(training_data,70, 30, 1.0, test_data=validation_data)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete
Epoch 10 complete
Epoch 11 complete
Epoch 12 complete
Epoch 13 complete
Epoch 14 complete
Epoch 15 complete
Epoch 16 complete
Epoch 17 complete
Epoch 18 complete
Epoch 19 complete
Epoch 20 complete
Epoch 21 complete
Epoch 22 complete
Epoch 23 complete
Epoch 24 complete
Epoch 25 complete
Epoch 26 complete
Epoch 27 complete
Epoch 28 complete
Epoch 29 complete
Epoch 30 complete
Epoch 31 complete
Epoch 32 complete
Epoch 33 complete
Epoch 34 complete
Epoch 35 complete
Epoch 36 complete
Epoch 37 complete
Epoch 38 complete
Epoch 39 complete
Epoch 40 complete
Epoch 41 complete
Epoch 42 complete
Epoch 43 complete
Epoch 44 complete
Epoch 45 complete
Epoch 46 complete
Epoch 47 complete
Epoch 48 complete
Epoch 49 complete
Epoch 50 complete
Epoch 51 complete
Epoch 52 complete
Epoch 53 complete
Epoch 54 complete
Epoch 55 complete
Ep

In [19]:
net.SGD(training_data,70, 30, 1.0, test_data=test_data)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete
Epoch 10 complete
Epoch 11 complete
Epoch 12 complete
Epoch 13 complete
Epoch 14 complete
Epoch 15 complete
Epoch 16 complete
Epoch 17 complete
Epoch 18 complete
Epoch 19 complete
Epoch 20 complete
Epoch 21 complete
Epoch 22 complete
Epoch 23 complete
Epoch 24 complete
Epoch 25 complete
Epoch 26 complete
Epoch 27 complete
Epoch 28 complete
Epoch 29 complete
Epoch 30 complete
Epoch 31 complete
Epoch 32 complete
Epoch 33 complete
Epoch 34 complete
Epoch 35 complete
Epoch 36 complete
Epoch 37 complete
Epoch 38 complete
Epoch 39 complete
Epoch 40 complete
Epoch 41 complete
Epoch 42 complete
Epoch 43 complete
Epoch 44 complete
Epoch 45 complete
Epoch 46 complete
Epoch 47 complete
Epoch 48 complete
Epoch 49 complete
Epoch 50 complete
Epoch 51 complete
Epoch 52 complete
Epoch 53 complete
Epoch 54 complete
Epoch 55 complete
Ep

In [7]:
net.SGD(training_data,70, 28, 1.1, test_data=test_data)

Epoch 0 : 5351 / 10000
Epoch 1 : 6491 / 10000
Epoch 2 : 7908 / 10000
Epoch 3 : 8010 / 10000
Epoch 4 : 8141 / 10000
Epoch 5 : 8228 / 10000
Epoch 6 : 8296 / 10000
Epoch 7 : 8212 / 10000
Epoch 8 : 8306 / 10000
Epoch 9 : 8314 / 10000
Epoch 10 : 8370 / 10000
Epoch 11 : 8403 / 10000
Epoch 12 : 8388 / 10000
Epoch 13 : 8409 / 10000
Epoch 14 : 8432 / 10000
Epoch 15 : 8389 / 10000
Epoch 16 : 8442 / 10000
Epoch 17 : 8472 / 10000
Epoch 18 : 8454 / 10000
Epoch 19 : 8481 / 10000
Epoch 20 : 8488 / 10000
Epoch 21 : 8487 / 10000
Epoch 22 : 8486 / 10000
Epoch 23 : 8506 / 10000
Epoch 24 : 8530 / 10000
Epoch 25 : 8562 / 10000
Epoch 26 : 8523 / 10000
Epoch 27 : 8536 / 10000
Epoch 28 : 8542 / 10000
Epoch 29 : 8535 / 10000
Epoch 30 : 8561 / 10000
Epoch 31 : 8535 / 10000
Epoch 32 : 8547 / 10000
Epoch 33 : 8523 / 10000
Epoch 34 : 8548 / 10000
Epoch 35 : 8545 / 10000
Epoch 36 : 8572 / 10000
Epoch 37 : 8577 / 10000
Epoch 38 : 8557 / 10000
Epoch 39 : 8594 / 10000
Epoch 40 : 8578 / 10000
Epoch 41 : 8576 / 10000
Ep

In [5]:
net.SGD(training_data,80, 28, 1.0, test_data=validation_data)

Epoch 0 : 4492 / 10000
Epoch 1 : 4593 / 10000
Epoch 2 : 4629 / 10000
Epoch 3 : 5391 / 10000
Epoch 4 : 5434 / 10000
Epoch 5 : 5916 / 10000
Epoch 6 : 6153 / 10000
Epoch 7 : 6218 / 10000
Epoch 8 : 6250 / 10000
Epoch 9 : 6209 / 10000
Epoch 10 : 6759 / 10000
Epoch 11 : 6865 / 10000
Epoch 12 : 6857 / 10000
Epoch 13 : 6893 / 10000
Epoch 14 : 6903 / 10000
Epoch 15 : 6918 / 10000
Epoch 16 : 6927 / 10000
Epoch 17 : 6947 / 10000
Epoch 18 : 6969 / 10000
Epoch 19 : 6967 / 10000
Epoch 20 : 6940 / 10000
Epoch 21 : 6981 / 10000
Epoch 22 : 6989 / 10000
Epoch 23 : 6994 / 10000
Epoch 24 : 7000 / 10000
Epoch 25 : 6987 / 10000
Epoch 26 : 7004 / 10000
Epoch 27 : 7023 / 10000
Epoch 28 : 7013 / 10000
Epoch 29 : 7012 / 10000
Epoch 30 : 7029 / 10000
Epoch 31 : 7031 / 10000
Epoch 32 : 7026 / 10000
Epoch 33 : 7029 / 10000
Epoch 34 : 7041 / 10000
Epoch 35 : 7029 / 10000
Epoch 36 : 7035 / 10000
Epoch 37 : 7030 / 10000
Epoch 38 : 7036 / 10000
Epoch 39 : 7053 / 10000
Epoch 40 : 7063 / 10000
Epoch 41 : 7043 / 10000
Ep

In [6]:
import time
start=time.time()
net = Network([784, 50, 10])
net.SGD(training_data,75, 10, 1.255, test_data=validation_data)
time.sleep(5)
end=time.time()
allt=end-start-5
print("用时：",allt)

3
Epoch 0 : 7253 / 10000
Epoch 1 : 8079 / 10000
Epoch 2 : 8202 / 10000
Epoch 3 : 8366 / 10000
Epoch 4 : 8414 / 10000
Epoch 5 : 8404 / 10000
Epoch 6 : 8476 / 10000
Epoch 7 : 8444 / 10000
Epoch 8 : 8535 / 10000
Epoch 9 : 8471 / 10000
Epoch 10 : 8567 / 10000
Epoch 11 : 8604 / 10000
Epoch 12 : 8599 / 10000
Epoch 13 : 8544 / 10000
Epoch 14 : 8597 / 10000
Epoch 15 : 8586 / 10000
Epoch 16 : 8618 / 10000
Epoch 17 : 8537 / 10000
Epoch 18 : 8602 / 10000
Epoch 19 : 8643 / 10000
Epoch 20 : 8608 / 10000
Epoch 21 : 8667 / 10000
Epoch 22 : 8612 / 10000
Epoch 23 : 8652 / 10000
Epoch 24 : 8660 / 10000
Epoch 25 : 8670 / 10000
Epoch 26 : 8675 / 10000
Epoch 27 : 8658 / 10000
Epoch 28 : 8687 / 10000
Epoch 29 : 8587 / 10000
Epoch 30 : 8659 / 10000
Epoch 31 : 8700 / 10000
Epoch 32 : 8706 / 10000
Epoch 33 : 8699 / 10000
Epoch 34 : 8677 / 10000
Epoch 35 : 8688 / 10000
Epoch 36 : 8703 / 10000
Epoch 37 : 8678 / 10000
Epoch 38 : 8699 / 10000
Epoch 39 : 8624 / 10000
Epoch 40 : 8741 / 10000
Epoch 41 : 8731 / 10000


In [5]:
import time
start=time.time()
net = Network([784, 50, 10])
net.SGD(training_data,75, 10, 1.26, test_data=test_data)
time.sleep(5)
end=time.time()
allt=end-start-5
print("用时：",allt)

3
Epoch 0 : 7455 / 10000
Epoch 1 : 7619 / 10000
Epoch 2 : 8241 / 10000
Epoch 3 : 8288 / 10000
Epoch 4 : 8340 / 10000
Epoch 5 : 8400 / 10000
Epoch 6 : 8418 / 10000
Epoch 7 : 8405 / 10000
Epoch 8 : 8467 / 10000
Epoch 9 : 8463 / 10000
Epoch 10 : 8505 / 10000
Epoch 11 : 8494 / 10000
Epoch 12 : 8540 / 10000
Epoch 13 : 8449 / 10000
Epoch 14 : 8543 / 10000
Epoch 15 : 8554 / 10000
Epoch 16 : 8546 / 10000
Epoch 17 : 8524 / 10000
Epoch 18 : 8523 / 10000
Epoch 19 : 8551 / 10000
Epoch 20 : 8581 / 10000
Epoch 21 : 8609 / 10000
Epoch 22 : 8577 / 10000
Epoch 23 : 8606 / 10000
Epoch 24 : 8598 / 10000
Epoch 25 : 8585 / 10000
Epoch 26 : 8602 / 10000
Epoch 27 : 8632 / 10000
Epoch 28 : 8645 / 10000
Epoch 29 : 8648 / 10000
Epoch 30 : 8640 / 10000
Epoch 31 : 8625 / 10000
Epoch 32 : 8634 / 10000
Epoch 33 : 8654 / 10000
Epoch 34 : 8628 / 10000
Epoch 35 : 8632 / 10000
Epoch 36 : 8668 / 10000
Epoch 37 : 8680 / 10000
Epoch 38 : 8684 / 10000
Epoch 39 : 8675 / 10000
Epoch 40 : 8628 / 10000
Epoch 41 : 8666 / 10000
